# Řešení kvízu ze třetí lekce

## Část 1

### Otázka 1

Napiš příkaz, kterým změníš označení třídy ve sloupci Pclass z čísel 1, 2 a 3 a "first", "second" and "third". Využij prosím metodu map.

In [7]:
import pandas as pd
data = pd.read_csv("titanic.csv")

categories = {1: "first", 2: "second", 3: "third"}
data["pclass"] = data["Pclass"].map(categories)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,pclass
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,third
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,first
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,third
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,first
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,third


## Část 2

Pro tuto část bude potřebovat data o amerických volbách, která najdeš zde.
Data obsahují počty hlasů jednotlivým kandidátům v prezidentských volbách v letech 1976 až 2020 pro jednotlivé státy Spojených států amerických. Důležité budou sloupce year (rok), state (stát), party_simplified (název strany, za které kandidát nebo kandidátka kandidoval(a)) a rank (pořadí v daném státě a daném roce).

### Otázka 2

Napiš dotaz, kterým ponecháš v tabulce data pouze o vítězi voleb v každém státu a v každém roce. Vítěze voleb v daném státu a daném roce poznáš podle toho, že ve sloupci rank má hodnotu 1.

In [8]:
data_usa = pd.read_csv("data_usa.csv")
data_usa = data_usa[data_usa["rank"] == 1] 
data_usa.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,rank
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT,1
7,1976,ALASKA,AK,2,94,81,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,71555,123574,20210113,NaN,REPUBLICAN,1
11,1976,ARIZONA,AZ,4,86,61,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,418642,742719,20210113,NaN,REPUBLICAN,1
19,1976,ARKANSAS,AR,5,71,42,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,498604,767535,20210113,NaN,DEMOCRAT,1
23,1976,CALIFORNIA,CA,6,93,71,US PRESIDENT,"FORD, GERALD",REPUBLICAN,False,3882244,7803770,20210113,NaN,REPUBLICAN,1


### Otázka 3

Pokračuj nyní v práci s filtrovanými daty z předchozí otázky. Přidej nový sloupec previous_winner_party, který bude obsahovat stranu vítěze voleb (sloupec party_simplified) v daném státě v předchozích volbách. Z toho bychom pak mohli zjistit, jak moc se mění vítěz voleb v jednotlivých státech. Ty, kde se vítěz mění, jsou označovány jako swing states a je jich menšina. Ve většině států volí lidé dlouhodobě kandidáty jedné strany (např. v Kansasu volí Republikány od roku 1968, v Kalifornii volí Demokraty od roku 1992 atd.).

Například pro stát ALABAMA a rok 1976 bude tento nový sloupec prázdný (volby 1976 jsou první, ke kterým máme data) a pro rok 1980 bude obsahovat hodnotu DEMOCRAT (v roce 1976 vyhráli demokraté).

Pozor na to, že před použitím metody shift je nutné data seřadit. Do řešení vloži i příkaz na řazení.

In [9]:
data_usa = data_usa.sort_values(["state", "year"])
data_usa["previous_winner_party"] = data_usa.groupby("state")["party_simplified"].shift(1)
data_usa.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,rank,previous_winner_party
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT,1,NaN
332,1980,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,654192,1341929,20210113,NaN,REPUBLICAN,1,DEMOCRAT
696,1984,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,872849,1441713,20210113,NaN,REPUBLICAN,1,REPUBLICAN
1019,1988,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,815576,1378476,20210113,NaN,REPUBLICAN,1,REPUBLICAN
1304,1992,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,804283,1688060,20210113,NaN,REPUBLICAN,1,REPUBLICAN


### Otázka 4

Podívej se na obrázek níže. Obsahuje data po aplikaci metody shift, která ale nebyl nastavená úplně stoprocentně.  Poslední sloupec  previous_winner_party obsahuje řešení předchozí otázky, ale je v něm jedna chyba.  Všimni si řádku s rokem 1976 a státem ALASKA. Zde by měla být prázdná hodnota, protože vítěze voleb v roce 1972 v datech nemáme, zde ale je vloženo REPUBLICAN. To se tam dostalo z roku 2020 ze státu ALABAMA, tj. "přelil" se nám vítěz z jednoho státu do druhého. To ale nechceme. Co bylo špatně v postupu, který vygeneroval data na obrázku?

![3.png](3.png)

*Správná odpověď je: Nebyla použitá metoda groupby. Data byla sařazena správně - nejdříve podle států a poté podle roků.*

In [14]:
# Toto je správně - u Aljašky u roku 1976 je prázdná hodnota
data_usa["previous_winner_party"] = data_usa.groupby("state")["party_simplified"].shift(1)
data_usa.head(20)

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,rank,previous_winner_party
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT,1,NaN
332,1980,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,654192,1341929,20210113,NaN,REPUBLICAN,1,DEMOCRAT
696,1984,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,872849,1441713,20210113,NaN,REPUBLICAN,1,REPUBLICAN
1019,1988,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,815576,1378476,20210113,NaN,REPUBLICAN,1,REPUBLICAN
1304,1992,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,804283,1688060,20210113,NaN,REPUBLICAN,1,REPUBLICAN
1674,1996,ALABAMA,AL,1,63,41,US PRESIDENT,"DOLE, ROBERT",REPUBLICAN,False,769044,1534349,20210113,NaN,REPUBLICAN,1,REPUBLICAN
2042,2000,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE W.",REPUBLICAN,False,941173,1666272,20210113,NaN,REPUBLICAN,1,REPUBLICAN
2411,2004,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE W.",REPUBLICAN,False,1176394,1883415,20210113,NaN,REPUBLICAN,1,REPUBLICAN
2728,2008,ALABAMA,AL,1,63,41,US PRESIDENT,"MCCAIN, JOHN",REPUBLICAN,False,1266546,2099819,20210113,NaN,REPUBLICAN,1,REPUBLICAN
3079,2012,ALABAMA,AL,1,63,41,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,1255925,2074338,20210113,NaN,REPUBLICAN,1,REPUBLICAN


In [15]:
# Nyní groupby vynecháme. U Aljašky je nyní hodnota, která by tam neměla být
data_usa["previous_winner_party"] = data_usa["party_simplified"].shift(1)
data_usa.head(20)

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,rank,previous_winner_party
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT,1,None
332,1980,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,654192,1341929,20210113,NaN,REPUBLICAN,1,DEMOCRAT
696,1984,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,872849,1441713,20210113,NaN,REPUBLICAN,1,REPUBLICAN
1019,1988,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,815576,1378476,20210113,NaN,REPUBLICAN,1,REPUBLICAN
1304,1992,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,804283,1688060,20210113,NaN,REPUBLICAN,1,REPUBLICAN
1674,1996,ALABAMA,AL,1,63,41,US PRESIDENT,"DOLE, ROBERT",REPUBLICAN,False,769044,1534349,20210113,NaN,REPUBLICAN,1,REPUBLICAN
2042,2000,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE W.",REPUBLICAN,False,941173,1666272,20210113,NaN,REPUBLICAN,1,REPUBLICAN
2411,2004,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE W.",REPUBLICAN,False,1176394,1883415,20210113,NaN,REPUBLICAN,1,REPUBLICAN
2728,2008,ALABAMA,AL,1,63,41,US PRESIDENT,"MCCAIN, JOHN",REPUBLICAN,False,1266546,2099819,20210113,NaN,REPUBLICAN,1,REPUBLICAN
3079,2012,ALABAMA,AL,1,63,41,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,1255925,2074338,20210113,NaN,REPUBLICAN,1,REPUBLICAN


### Otázka 5

Přidal sloupec percentage, který bude obsahovat počet procent hlasů pro vítězného kandidáta v rámci daného státu. Hodnotu získáš tak, že vydělíš sloupec candidatevotes sloupcem totalvotes.
Je teoreticky možné použít metodu apply, ale v tomto případě je to zbytečné a je jednodušší využít počítaný sloupec, který budeš znát z předchozích kurzů (např. Python pro data 1, Python 1 atd.). Případně se podívej na tuto lekci.

In [16]:
data_usa["percentage"] = data_usa["candidatevotes"] / data_usa["totalvotes"]
data_usa.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,rank,previous_winner_party,percentage
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT,1,None,0.557273
332,1980,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,654192,1341929,20210113,NaN,REPUBLICAN,1,DEMOCRAT,0.487501
696,1984,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,872849,1441713,20210113,NaN,REPUBLICAN,1,REPUBLICAN,0.605425
1019,1988,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,815576,1378476,20210113,NaN,REPUBLICAN,1,REPUBLICAN,0.591650
1304,1992,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,804283,1688060,20210113,NaN,REPUBLICAN,1,REPUBLICAN,0.476454


## Část 3

### Otázka 6

Přidej nový sloupec vzdalenost_od_nadrazi, která bude obsahovat vzdálenost od nádraží Opava Východ v kilometrech. Jako zeměpisnou polohu nádraží použij zeměpisnou šířku 49.9345092 a zeměpisnou délku 17.9085369. Následně vytvoř funkci vzdalenost_od_nadrazi, která pro všechna sportoviště spočte vzdálenost od nádraží. Pro výpočet je potřeba si nainstalovat modul geopy (klasicky přes pip).

In [17]:
import pandas as pd
from geopy import distance

data = pd.read_json("sportoviste.json")

def vzdalenost_od_nadrazi(row):
    souradnice_nadrazi = [49.9345092, 17.9085369]
    row = row[["POINT_Y", "POINT_X"]]
    return distance.distance(row, souradnice_nadrazi).km

data["vzdalenost_od_nadrazi"] = data.apply(vzdalenost_od_nadrazi, axis=1)
data.head()

,OBJECTID,dostupnost,prostor,vlastnik,spravce,foto1,foto2,provozovatel_ic,provozovatel_nazev,sportoviste,...,NEAR_FID,NEAR_DIST,NEAR_X,NEAR_Y,NEAR_ANGLE,cena,foto3,POINT_X,POINT_Y,vzdalenost_od_nadrazi
0,1,volně přístupné,venkovní,statutární město Opava,Technické služby Opava,None,None,64618188,Technické služby Opava s.r.o.,sportoviště,...,7167.0,63.858786,-497883.88,-1086130.13,-171.946563,NaN,NaN,17.886344,49.952289,2.539389
1,2,po domluvě / v rámci otevírací doby,venkovní,statutární město Opava,Seniorcentrum Opava,None,None,71196943,Seniorcentrum Opava,sportoviště,...,2483.0,22.821850,-496032.44,-1087009.24,-66.872318,NaN,NaN,17.912125,49.945973,1.300870
2,3,volně přístupné,venkovní,statutární město Opava,Technické služby Opava,None,None,64618188,Technické služby Opava s.r.o.,sportoviště,...,7104.0,47.868888,-497620.28,-1087329.31,147.833651,NaN,NaN,17.891244,49.941440,1.461320
3,4,volně přístupné,venkovní,statutární město Opava,Technické služby Opava,None,None,64618188,Technické služby Opava s.r.o.,dětské hřiště,...,6559.0,35.343983,-497592.61,-1087410.36,-156.606515,NaN,NaN,17.891568,49.941084,1.420832
4,5,volně přístupné,venkovní,statutární město Opava,Technické služby Opava,None,None,64618188,Technické služby Opava s.r.o.,dětské hřiště,...,3765.0,33.956871,-496361.26,-1087028.91,174.000780,NaN,NaN,17.908212,49.945345,1.205469


### Otázka 7

Nyní vytvoř funkci vzdalenost_od_bodu, která bude umět spočítat obecnou vzdálenost od libovolného bodu. Funkce bude mít druhý parametr - bod, který bude obsahovat seznam, ve kterém bude na pozici 0 zeměpisná šířka bodu a na pozici 1 zeměpisná délka nějakého bodu. Dále použij tuto funkci v kombinaci s metodou apply, které předej souřadnice nádraží s využitím parametru args.

In [20]:
def vzdalenost_od_bodu(row, libovolny_bod):
    row = row[["POINT_Y", "POINT_X"]]
    return distance.distance(row, libovolny_bod).km

vybrany_bod = [49.9345092, 17.9085369]
data["vzdalenost_od_bodu"] = data.apply(vzdalenost_od_bodu, axis=1, args=[vybrany_bod])
data.head()

,OBJECTID,dostupnost,prostor,vlastnik,spravce,foto1,foto2,provozovatel_ic,provozovatel_nazev,sportoviste,...,NEAR_DIST,NEAR_X,NEAR_Y,NEAR_ANGLE,cena,foto3,POINT_X,POINT_Y,vzdalenost_od_nadrazi,vzdalenost_od_bodu
0,1,volně přístupné,venkovní,statutární město Opava,Technické služby Opava,None,None,64618188,Technické služby Opava s.r.o.,sportoviště,...,63.858786,-497883.88,-1086130.13,-171.946563,NaN,NaN,17.886344,49.952289,2.539389,2.539389
1,2,po domluvě / v rámci otevírací doby,venkovní,statutární město Opava,Seniorcentrum Opava,None,None,71196943,Seniorcentrum Opava,sportoviště,...,22.821850,-496032.44,-1087009.24,-66.872318,NaN,NaN,17.912125,49.945973,1.300870,1.300870
2,3,volně přístupné,venkovní,statutární město Opava,Technické služby Opava,None,None,64618188,Technické služby Opava s.r.o.,sportoviště,...,47.868888,-497620.28,-1087329.31,147.833651,NaN,NaN,17.891244,49.941440,1.461320,1.461320
3,4,volně přístupné,venkovní,statutární město Opava,Technické služby Opava,None,None,64618188,Technické služby Opava s.r.o.,dětské hřiště,...,35.343983,-497592.61,-1087410.36,-156.606515,NaN,NaN,17.891568,49.941084,1.420832,1.420832
4,5,volně přístupné,venkovní,statutární město Opava,Technické služby Opava,None,None,64618188,Technické služby Opava s.r.o.,dětské hřiště,...,33.956871,-496361.26,-1087028.91,174.000780,NaN,NaN,17.908212,49.945345,1.205469,1.205469
